# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Импортируем нужные нам библиотеки.

In [1]:
import pandas as pd

from numpy.random import RandomState

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error

from scipy import stats as st
from scipy.stats import t

import matplotlib.pyplot as plt

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

data_0 = data_0.drop(['id'], axis=1)
data_1 = data_1.drop(['id'], axis=1)
data_2 = data_2.drop(['id'], axis=1)

print(data_0.corr(method='spearman'))
print(data_1.corr(method='spearman'))
print(data_2.corr(method='spearman'))

               f0        f1        f2   product
f0       1.000000 -0.471395 -0.002685  0.128417
f1      -0.471395  1.000000  0.001413 -0.181143
f2      -0.002685  0.001413  1.000000  0.486394
product  0.128417 -0.181143  0.486394  1.000000
               f0        f1        f2   product
f0       1.000000  0.182248 -0.002000 -0.122869
f1       0.182248  1.000000 -0.003678 -0.033908
f2      -0.002000 -0.003678  1.000000  0.975605
product -0.122869 -0.033908  0.975605  1.000000
               f0        f1        f2   product
f0       1.000000  0.002493  0.000053 -0.002464
f1       0.002493  1.000000  0.000378 -0.001463
f2       0.000053  0.000378  1.000000  0.448463
product -0.002464 -0.001463  0.448463  1.000000


Проверим количество пропущенных значений в данных

In [3]:
print(data_0.isna().sum(), '\n', data_1.isna().sum(), '\n', data_2.isna().sum())

f0         0
f1         0
f2         0
product    0
dtype: int64 
 f0         0
f1         0
f2         0
product    0
dtype: int64 
 f0         0
f1         0
f2         0
product    0
dtype: int64


In [4]:
print(data_0.describe(),'\n', data_1.describe(),'\n', data_2.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347 
                   f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%     

Разобьем данные каждого региона на обучающую и валидационные выборки в соотношении 75:25.

In [5]:
features_0 = data_0.drop(['product'], axis=1)
target_0 = data_0['product']

features_1 = data_1.drop(['product'], axis=1)
target_1 = data_1['product']

features_2 = data_2.drop(['product'], axis=1)
target_2 = data_2['product']

In [6]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = (
    train_test_split(features_0,
                     target_0,
                     test_size=.25,
                     random_state=12345)
)

features_train_1, features_valid_1, target_train_1, target_valid_1 = (
    train_test_split(features_1,
                     target_1,
                     test_size=.25,
                     random_state=12345)
)

features_train_2, features_valid_2, target_train_2, target_valid_2 = (
    train_test_split(features_2,
                     target_2,
                     test_size=.25,
                     random_state=12345)
)

Проведем масштабирование данных методом StandardScaler.

In [7]:
scaler = StandardScaler(with_mean=False)
features_train_0 = scaler.fit_transform(features_train_0)
features_valid_0 = scaler.transform(features_valid_0)

features_train_1 = scaler.fit_transform(features_train_1)
features_valid_1 = scaler.transform(features_valid_1)

features_train_2 = scaler.fit_transform(features_train_2)
features_valid_2 = scaler.transform(features_valid_2)

print(features_train_0.shape)
print(features_valid_0.shape)

print(features_train_1.shape)
print(features_valid_1.shape)

print(features_train_2.shape)
print(features_valid_2.shape)

(75000, 3)
(25000, 3)
(75000, 3)
(25000, 3)
(75000, 3)
(25000, 3)


Так как у нас нет категориальных признаков, прямое кодирование данных нам не понадобится, но так как пристутсвуют количественные признаки с разными разбросами значений, данные были промасштабированны методом StandardScaler.

После обработки и изучения исходных данных, можно заметить высокую положительную корреляцию у всех трех датасетов между целевым признаком product и признаком f2, так же можно отметить что датасеты оказались без пропущенных значений, потому было произведено лишь масштабирование данных методом StandardScaler.

## Обучение и проверка модели

Обучим модели для каждого региона по отдельности.

### Регион 1.

Обучим модель LinearRegressor.

In [8]:
model_LR_0 = LinearRegression()
model_LR_0.fit(features_train_0, target_train_0)
predicted_valid_0 = model_LR_0.predict(features_valid_0)

print(f'Средний запас предсказанного сырья: {predicted_valid_0.mean()}\n\
RMSE модели: {mean_squared_error(target_valid_0, predicted_valid_0) ** .5}')

Средний запас предсказанного сырья: 92.59256778438035
RMSE модели: 37.5794217150813


### Регион 2.

Обучим модель LinearRegressor.

In [9]:
model_LR_1 = LinearRegression()
model_LR_1.fit(features_train_1, target_train_1)
predicted_valid_1 = model_LR_1.predict(features_valid_1)

print(f'Средний запас предсказанного сырья: {predicted_valid_1.mean()}\n\
RMSE модели: {mean_squared_error(target_valid_1, predicted_valid_1) ** .5}')

Средний запас предсказанного сырья: 68.72854689544602
RMSE модели: 0.893099286775617


### Регион 3.

Обучим модель LinearRegressor.

In [18]:
model_LR_2 = LinearRegression()
model_LR_2.fit(features_train_2, target_train_2)
predicted_valid_2 = model_LR_2.predict(features_valid_2)

print(f'Средний запас предсказанного сырья: {predicted_valid_2.mean()}\n\
RMSE модели: {mean_squared_error(target_valid_2, predicted_valid_2) ** .5}')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.48 µs
Learning rate set to 0.080991
0:	learn: 43.8723871	total: 8.95ms	remaining: 8.94s
1:	learn: 43.1477505	total: 17.9ms	remaining: 8.94s
2:	learn: 42.5058208	total: 26.8ms	remaining: 8.9s
3:	learn: 41.9403960	total: 35.6ms	remaining: 8.86s
4:	learn: 41.4251580	total: 44ms	remaining: 8.75s
5:	learn: 40.9487480	total: 52.4ms	remaining: 8.68s
6:	learn: 40.5925540	total: 60.5ms	remaining: 8.59s
7:	learn: 40.2571614	total: 68.3ms	remaining: 8.47s
8:	learn: 39.9359521	total: 76.3ms	remaining: 8.4s
9:	learn: 39.6886507	total: 84.3ms	remaining: 8.35s
10:	learn: 39.4638430	total: 92.2ms	remaining: 8.29s
11:	learn: 39.2474091	total: 100ms	remaining: 8.24s
12:	learn: 39.0687455	total: 108ms	remaining: 8.19s
13:	learn: 38.8993734	total: 116ms	remaining: 8.13s
14:	learn: 38.7526168	total: 124ms	remaining: 8.11s
15:	learn: 38.6407326	total: 132ms	remaining: 8.1s
16:	learn: 38.5375902	total: 140ms	remaining: 8.09s
17:	learn: 38.4429433	tota

Как видно по результатам выше, в 0 и 2 регионах модели предсказали самые большие средние запасы сырья, однако, также следует отметить и высокий показатель RMSE в этих регионах, вероятно это связано с выкоким разбросом значений объемов сырья.

Так как модели DecisionTree и RandomForest имеют недостаточную предсказуемость, будем использовать только модели линейной регрессии. 

## Подготовка к расчёту прибыли

Сохраним все ключевые значения для рассчетов в отдельных переменных и подсчитаем минимальный объем сырья для безубыточной разработки месторождений.

In [11]:
UNIT_PROFIT = 450_000
BUDGET = 10_000_000_000
NUM_WELLS = 200

BREAKEVEN = BUDGET / UNIT_PROFIT / NUM_WELLS
print(f'Минимальный объем сырья для безубыточной разработки месторождений: {BREAKEVEN}')

Минимальный объем сырья для безубыточной разработки месторождений: 111.11111111111111


Так как мы ищем 200 самых прибыльных скважин, следовательно далее будем смотреть для каких регионов общий средний объем их 200 топовых скважин сырья больше минимального 111.(11)

Напишем функцию рассчета прибыли для каждого отдельного региона.

In [12]:
STATE = RandomState(12345)

def profit_calc(predicted, target):
    count = 200
    n = 500
    predicted = pd.Series(predicted, name='predicted').reset_index()
    target = pd.Series(target, name='target').reset_index()

    target_predicted = pd.concat([predicted, target], axis=1).drop(['index', 'index'], axis=1)
    
    predicted_sampled = target_predicted.sample(n=n, replace=True, random_state=STATE)
    predicted_sampled = predicted_sampled.sort_values(by='predicted', ascending=False)[:count]
    profit = predicted_sampled['target'].sum() * UNIT_PROFIT - BUDGET
    
    return profit

## Расчёт прибыли и рисков 

### Регион 0.

Рассчитаем прибыль и риски для региона 0.

In [13]:
values = []

for i in range(1000):
    profit = profit_calc(predicted_valid_0, target_valid_0)
    values.append(profit)

values = pd.Series(values)

confidence_interval = t.interval(.975, len(values) - 1, values.mean() / 200, values.sem())
upper = values.quantile(.975, interpolation='linear')
lower = values.quantile(.025, interpolation='linear')

print(f'Средняя прибыль: {values.mean()}\n\
97.5% доверительный интервал прибыли: {confidence_interval}\n\
97.5% квантиль: {upper}\n\
2.5% квантиль: {lower}\n\
Риск: {(values < 0).mean():.2%}')

Средняя прибыль: 396164984.8023711
97.5% доверительный интервал прибыли: (-16945163.17950204, 20906813.027525768)
97.5% квантиль: 909766941.5534226
2.5% квантиль: -111215545.89049526
Риск: 6.90%


У региона 0 можно отметить высокие показатели средней прибыли и доверительного интервала, однако восоко и значение риска.

### Регион 1.

Рассчитаем прибыль и риски для региона 1.

In [14]:
values = []

for i in range(1000):
    profit = profit_calc(predicted_valid_1, target_valid_1)
    values.append(profit)

values = pd.Series(values)

confidence_interval = t.interval(.975, len(values) - 1, values.mean() / 200, values.sem())
upper = values.quantile(.975, interpolation='linear')
lower = values.quantile(.025, interpolation='linear')

print(f'Средняя прибыль: {values.mean()}\n\
97.5% доверительный интервал прибыли: {confidence_interval}\n\
97.5% квантиль: {upper}\n\
2.5% квантиль: {lower}\n\
Риск: {(values < 0).mean():.2%}')

Средняя прибыль: 461155817.2772397
97.5% доверительный интервал прибыли: (-11795342.553724157, 16406900.726496564)
97.5% квантиль: 862952060.2637234
2.5% квантиль: 78050810.7517417
Риск: 0.70%


В регионе 1 можно отметить самые высокие показатели средней прибыли и доверительного интервала, и при этом у него же и самое низкое значение риска.

### Регион 2.

Рассчитаем прибыль и риски для региона 2.

In [15]:
values = []

for i in range(1000):
    profit = profit_calc(predicted_valid_2, target_valid_2)
    values.append(profit)

values = pd.Series(values)

confidence_interval = t.interval(.975, len(values) - 1, values.mean() / 200, values.sem())
upper = values.quantile(.975, interpolation='linear')
lower = values.quantile(.025, interpolation='linear')

print(f'Средняя прибыль: {values.mean()}\n\
97.5% доверительный интервал прибыли: {confidence_interval}\n\
97.5% квантиль: {upper}\n\
2.5% квантиль: {lower}\n\
Риск: {(values < 0).mean():.2%}')

Средняя прибыль: 688222405.4773269
97.5% доверительный интервал прибыли: (-15277290.54517047, 22159514.599943753)
97.5% квантиль: 1226033239.9178598
2.5% квантиль: 181980498.7215565
Риск: 0.50%


В регионе 2 можно отметить наименьшие среднюю прибыль и доверительный интервал, значение риска так же сильно превышает пороговое.

## Общий вывод

Произведя рассчеты прибыли и рисков для каждого региона, можно рекомендовать для разработки регион 1 ввиду высокой средней прибыли, хорошие значения t-статистики и при этом минимального риска. Регион 0 и регион 2 не рекомендуются для разработки, так как в этих регионах высок риск убытков.

В ходе исследования были пройдены следующие шаги:
- Изучен исходный датасет.
- Предобработаны признаки.
- Выборки были разбиты на обучающую и валидационную.
- Проведено масштабирование данных методом StandardScaler.
- Обучены модели LinearRegression для каждого региона.
- Написана функция рассчета прибыли для каждого региона.
- Техникой Bootstrap были рассчитаны прибыль и риски для каждого региона.
- Написаны рекомендации к разработке меторождений в регионах.